# YT Comments analysis

In [2]:
from googleapiclient.discovery import build
import pandas as pd
from api import API_KEY

channel_id = "UCWeg2Pkate69NFdBeuRFTAw" #Squeezie channel
etoiles = 'UCABf02qOye7XYapcK1M45LQ'
youtube = build('youtube', 'v3', developerKey=API_KEY)
exemple_video = "qCKyRhkhqoQ"

In [3]:
class VideoRequest:
    """ Class VideoRequest can be modified instead of re-creating one from scratch """
    def __init__(self, part=None, id=None, chart=None, regionCode=None, maxResults=None, pageToken=None):
        self.part = part
        self.id = id
        self.chart = chart
        self.regionCode = regionCode
        self.maxResults = maxResults
        self.pageToken = pageToken
        
    def execute(self):
        request = youtube.videos().list(
            part = self.part,
            id = self.id,
            chart = self.chart,
            regionCode = self.regionCode,
            maxResults = self.maxResults,
            pageToken = self.pageToken
        )
        return request.execute()  

In [4]:
def get_channel_stats(youtube, channel_id):
    """ request by id to get most important channel stats """
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics,topicDetails",
        id=channel_id
    )
    response = request.execute()
    items = response.get('items', [])[0]
    data = {
        "channel_name": items.get('snippet', {}).get('title'),
        "channel_id": items.get('id'),
        "country": items.get('snippet', {}).get('country',""),
        "stats": items.get('statistics'),
        "topics": [wikilink.split('/')[-1] for wikilink in items.get('topicDetails', {}).get('topicCategories', [])],
    }
    del data['stats']['hiddenSubscriberCount']
    return data
    

get_channel_stats(youtube, channel_id)

{'channel_name': 'SQUEEZIE',
 'channel_id': 'UCWeg2Pkate69NFdBeuRFTAw',
 'country': 'FR',
 'stats': {'viewCount': '10439394851',
  'subscriberCount': '18700000',
  'videoCount': '1621'},
 'topics': ['Television_program', 'Entertainment']}

In [5]:
from datetime import datetime
def get_video_info(youtube, video_Id):
    request = VideoRequest(
        part="snippet,contentDetails,statistics,topicDetails",
        id=video_Id,
    )
    response = request.execute()
    items = response.get('items', [])[0]
    data = {
            "title": items.get('snippet', {}).get('title'),
            "id": items.get('id'),
            "publishedAt": items.get('snippet', {}).get('publishedAt'),
            "duration" : items.get('contentDetails').get('duration'),
            "ViewCount" : items.get('statistics', {}).get('viewCount'),
            "likeCount" : items.get('statistics', {}).get('likeCount'),
            "commentCount" : items.get('statistics', {}).get('commentCount'),  
            "tags" : items.get('snippet', {}).get('tags')
    }
    
    return data

# time = get_video_info(youtube, 'JRBGBjaR9Wg').get("publishedAt")
# print(datetime.strptime(time, '%Y-%m-%dT%H:%M:%SZ'))
get_video_info(youtube, 'JRBGBjaR9Wg')

{'title': "L'âge d'or de la piraterie dans l'Océan Indien",
 'id': 'JRBGBjaR9Wg',
 'publishedAt': '2024-02-16T11:00:22Z',
 'duration': 'PT20M9S',
 'ViewCount': '196172',
 'likeCount': '9433',
 'commentCount': '402',
 'tags': ['nota',
  'bene',
  'histoire',
  'ubisoft',
  'skull',
  'and',
  'bones',
  'pirate',
  'corsaire',
  'boucanier',
  'marin',
  'marine',
  'bateau',
  'navire',
  'capitaine',
  'inde',
  'mer',
  'caraïbes',
  'arabe',
  'chine',
  'france',
  'angleterre',
  'portugal',
  'âge',
  'or',
  'océan',
  'indien',
  'madagascar',
  'sainte-marie',
  'île',
  'réunion',
  'bourbon',
  'canon']}

In [7]:
def get_Most_Popular_Video_id(youtube, region:str):
    request = VideoRequest(
        part="snippet,contentDetails,statistics,topicDetails",
        chart="mostPopular",
        regionCode=region,
        maxResults=100,
        pageToken=''
    )
    response = request.execute()
    
    pages = [response]
    while response.get('nextPageToken'):
        request.pageToken = response.get('nextPageToken')
        response = request.execute()
        pages.append(response)
     
    
    id_top_video = [videos.get('id') for page in pages for videos in page.get('items')]
    return id_top_video

most_popular_id = get_Most_Popular_Video_id(youtube, 'FR')
print("=============")

top_vids = [get_video_info(youtube, id)['title'] for id in most_popular_id]
for vid in top_vids:
    print(vid)

200
On survit 24h sur une île déserte avec Mastu et Maxime Biaggi
LE SSERAFIM (르세라핌) 'EASY' OFFICIAL MV
Les IMAGES CACHÉES dans les DESSINS ANIMÉS (Le Roi Lion, Pokémon, La Petite Sirène...)
GAUCHE GAUCHE DROITE DROITE : LE JEU (avec Freddy, Anis & un guest surprise qui les a tués de rire)
Les premières vacances des Croûtons pour MJ, c'est partiiii ! 🏝️
LA FAQ DE LA VÉRITÉ (en me lissant les cheveux)
🤯LE PSG 24 - 25 (SANS MBAPPÉ) VA ÊTRE INCROYABLE!
Je traverse la frontière de la Corée du Nord ! - Episode Final
Construire Un Igloo Sans Expérience (pour y passer la nuit...) 🥶
TOUS les PAPAS DISNEY, du pire au meilleur !
JEUX EN VRAC - No One Lives Forever
UNE NUIT DANS LA SUITE LA PLUS CHÈRE DE FRANCE (avec Kalash Criminel)
CHOC DES GÉNÉRATIONS (Ft. Jean Reno, Chantal Ladesou et Kev Adams)
Jusqu'ici tout va bien - La semaine de Naïm
Carte Joker de Jérôme Commandeur - LOL : Qui rit, sort ! Saison 4 | Prime Video
24H DANS LA VOITURE LA PLUS PETITE DU MONDE
Partir prendre l'air (seule

In [15]:
def get_comment_id(youtube,video_Id):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults=100,
        pageToken = 'Z2V0X25ld2VzdF9maXJzdC0tQ2dnSWdBUVZGN2ZST0JJRkNJa2dHQUFTQlFpZElCZ0JFZ1VJcUNBWUFCSUZDSWNnR0FBU0JRaUlJQmdBR0FBaURRb0xDTGIxeHE0R0VOalkybmM'
    )
    response = request.execute()
    print(response)
    id=[]
    for i in range(len(response['items'])):       
            id.append(response['items'][i]["id"])
          
    
    return id

get_comment_id(youtube, 'JRBGBjaR9Wg')

{'kind': 'youtube#commentThreadListResponse', 'etag': 'Vyhnbfz7lWeqa2Bw1tqZXiyIE34', 'nextPageToken': 'Z2V0X25ld2VzdF9maXJzdC0tQ2dnSWdBUVZGN2ZST0JJRkNLZ2dHQUFTQlFpSElCZ0FFZ1VJblNBWUFSSUZDSWtnR0FBU0JRaUlJQmdBR0FBaURnb01DUHJId3E0R0VKRE15ZFFC', 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20}, 'items': [{'kind': 'youtube#commentThread', 'etag': 'w6iM5fW8M0iOCxfy6mMN99vr_Ag', 'id': 'Ugyio6-iU_xsNUYPvn94AaABAg', 'snippet': {'channelId': 'UCP46_MXP_WG_auH88FnfS1A', 'videoId': 'JRBGBjaR9Wg', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'U8vhzQjqzy_OXuwMEDpmraTdXPU', 'id': 'Ugyio6-iU_xsNUYPvn94AaABAg', 'snippet': {'channelId': 'UCP46_MXP_WG_auH88FnfS1A', 'videoId': 'JRBGBjaR9Wg', 'textDisplay': 'Bonjour je souhaiterais tester le Skull and Bones il y a des clés a gagné? Si oui je participe.', 'textOriginal': 'Bonjour je souhaiterais tester le Skull and Bones il y a des clés a gagné? Si oui je participe.', 'authorDisplayName': '@raffboss33', 'authorProfileImageUrl': 'https://yt3.

['Ugyio6-iU_xsNUYPvn94AaABAg',
 'Ugyd8HhsX_lrg5YZx9t4AaABAg',
 'Ugypv5hHdt-60U59FLx4AaABAg',
 'UgwkjgcJNHc9C_g1zDN4AaABAg',
 'UgxlGQhgm1FxPX0o9R54AaABAg',
 'Ugz4-1ufPF7pXE_0V1N4AaABAg',
 'Ugz6BtZ8FwKLwIznL0p4AaABAg',
 'UgyzM9HhoKglO48BKLl4AaABAg',
 'Ugyvz4ZPhcognBFGe2l4AaABAg',
 'UgwPNGeSP9WdThtm5v94AaABAg',
 'UgwLV3l9ww_4vC0Mm8x4AaABAg',
 'UgyIV0r1MPB-WMmljvB4AaABAg',
 'UgxHSW6CCm-6Jw4U9Dl4AaABAg',
 'Ugy4J2M0srMyUnxl9mx4AaABAg',
 'Ugy3T2wGYF6MCW0dhtx4AaABAg',
 'Ugwr1aQXpxF7kBrpmbF4AaABAg',
 'UgzulJhUsFUCm8BwCYt4AaABAg',
 'UgxbYC2Y4pPDU6Dt2v94AaABAg',
 'UgwKrmzfMkOBqu5VL-B4AaABAg',
 'UgxTFGfzhvW0VRo1NnJ4AaABAg']

In [ ]:
def get_comment(youtube,comment_id):
    request = youtube.comments().list(
        part="snippet",
        id=comment_id,
    )
    response = request.execute()
    comment=[]
    viewerRating=[]
    likeCount=[]
    publishedAt=[]
    for i in range(len(response['items'])):       
            comment.append(response['items'][i]['snippet']['textOriginal'])
            viewerRating.append(response['items'][i]['snippet']['viewerRating'])
            likeCount.append(response['items'][i]['snippet']['likeCount'])
            publishedAt.append(response['items'][i]['snippet']['publishedAt'])
    data = {
        "comment":comment,
        "viewerRating":viewerRating,
        "likeCount":likeCount,
        "publishedAt":publishedAt,
        }
    
    return data

In [ ]:
def get_all_comment(youtube,videoId):
    id=get_comment_id(youtube,videoId)
    for i in id : 
        print(get_comment(youtube,i))
        

In [ ]:

def get_video_info_top_50(youtube):
    top_50_id=get_Most_Popular_Video_id(youtube)
    print(top_50_id)
    for i in top_50_id :
        print(get_video_info(youtube,i))
        

In [ ]:
#print(get_video_comments(youtube,exemple_video))

print(get_comment_id(youtube,exemple_video))
get_all_comment(youtube,exemple_video)

In [ ]:
get_channel_stats(youtube, "UCWeg2Pkate69NFdBeuRFTAw")
print("ok")
print(get_video_info(youtube,exemple_video))
get_video_info_top_50(youtube)